In [ ]:
!pip install beautifulsoup4 openpyxl pandas


  Using cached beautifulsoup4-4.14.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)

   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]



In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

# Funkcja do pobierania danych z tabeli na stronie Bondspot
def get_bondspot_data():
    start_date = datetime(2024, 1, 1)
    end_date = datetime.today()
    print(start_date, end_date)
    all_data = []
    
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y%m%d")
        year = current_date.year
        month = current_date.strftime("%m")
        
        url = f'https://www.bondspot.pl/fixing_obligacji?date={formatted_date}&month={month}&year={year}&type=2'
        response = requests.get(url)
        
        if response.status_code == 200:
            print(f'Response correct for {formatted_date}')
            soup = BeautifulSoup(response.text, 'html.parser')
            tables = soup.find_all('table', {'class': 'tab_01'})
            if len(tables) > 1:
                table = tables[1]
                
                headers = [th.text.strip() for th in table.find('thead').find_all('th')]
                
                rows = table.find('tbody').find_all('tr')
                for row in rows:
                    columns = row.find_all('td')
                    if len(columns) > 0:
                        data = {headers[i]: columns[i].text.strip() for i in range(len(columns))}

                        data['Date'] = current_date.strftime("%Y-%m-%d")
                        
                        all_data.append(data)
        
        current_date += timedelta(days=1)
    
    df = pd.DataFrame(all_data)
    
    return df


df_bonds = get_bondspot_data()
print(df_bonds.head())
df_bonds.to_csv('bondspot_data_2024.csv', index=False)


2026-02-16 00:00:00 2026-02-18 20:51:07.800724
Response correct for 20260216
Response correct for 20260217
Response correct for 20260218
  LP   Nazwa          ISIN  Cena K  Cena S Rent.K Rent.S Cena fix Rent fix  \
0  1  DS0726  PL0000108866   99.66   99.85   3.25   2.81    99.76     3.02   
1  2  PS1026  PL0000113460   98.02   98.21   3.21   2.92    98.12     3.06   
2  3  WZ1126  PL0000113130  100.48  100.59      -      -   100.54        -   
3  4  OK0127  PL0000117289   97.14   97.28   3.15   2.99    97.21     3.07   
4  5  PS0527  PL0000114393  100.65  100.75   3.21   3.13   100.70     3.17   

         Date  
0  2026-02-16  
1  2026-02-16  
2  2026-02-16  
3  2026-02-16  
4  2026-02-16  
